# RBM and CRBM

Objective: Implement CRBM in Julia for time series analysis

In [1]:
# Import Distributions to generate the W matrix of the RBM
using Distributions
using MNIST
using Benchmarks

LoadError: ArgumentError: Module Benchmarks not found in current path.
Run `Pkg.add("Benchmarks")` to install the Benchmarks package.

In [2]:
type RBM{T <: Real}
    n_vis::Int
    n_hid::Int
    W::Matrix{T}         
    vis_bias::Vector{T}     
    hid_bias::Vector{T}   
    trained::Bool
end

function Base.show{T}(io::IO, rbm::RBM{T})
    n_vis = size(rbm.vis_bias, 1)
    n_hid = size(rbm.hid_bias, 1)
    trained = rbm.trained
    print(io, "RBM{$T}(n_vis=$n_vis, n_hid=$n_hid, trained=$trained)")
end

In [3]:
function sigmoid(vector::Array{Float64})
    return 1./(1 + e.^(-vector))
end

sigmoid (generic function with 1 method)

In [4]:
function initialize_RBM(n_vis, n_hid, sigma, T)
    
    return RBM{T}( n_vis,                                 # num visible units 
                   n_hid,                                 # num hidden unnits
                   rand(Normal(0,sigma), n_hid, n_vis),  # weight matrix
                   zeros(n_vis),                          # visible vector  
                   zeros(n_hid),                          # Hidden vector
                   false)                                 # trained
end

initialize_RBM (generic function with 1 method)

In [5]:
rbm = initialize_RBM(784, 225, 0.01, Float64)

RBM{Float64}(n_vis=784, n_hid=225, trained=false)

In [6]:
size(rbm.W)

(225,784)

In [9]:
X_train, y_train = MNIST.traindata()
X_test, y_test = MNIST.testdata()

T = Float32
X_train = Array{T}( (X_train - minimum(X_train))/(maximum(X_train) - minimum(X_train)) )
y_train = Array{T}(y_train)
X_test = Array{T}(X_test - minimum(X_test))/(maximum(X_test) - minimum(X_test)) 
y_test = Array{T}(y_test);

In [10]:
size(X_train_col)

LoadError: UndefVarError: X_train_col not defined

In [11]:
size(rbm.W)

(225,784)

In [12]:
X_batch = X_train_col[:,1:25];
size(X_batch)

LoadError: UndefVarError: X_train_col not defined

In [13]:
size(rbm.W * X_batch .+ rbm.hid_bias)

LoadError: UndefVarError: X_batch not defined

In [14]:
function contrastive_divergence_K(Xbatch, rbm, K::Integer, lr::Real)
        
    batch_size = size(Xbatch)[2]
    Delta_W = zeros(size(rbm.W))
    Delta_b = zeros(size(rbm.vis_bias))
    Delta_c = zeros(size(rbm.hid_bias))
    xneg = zeros(size(rbm.vis_bias))
    print("\nbatch size: ", batch_size)

    for i in 1:batch_size
        #print("\nx :",size(Xbatch[:,i]))
        #print("\nxneg :", size(xneg))
        #print("\nhneg :",size(hneg))

        x =  @view Xbatch[:,i]
        xneg = Xbatch[:,i]

        for k in 1:K
            hneg = sigmoid( rbm.W * xneg .+ rbm.hid_bias) .> rand(rbm.n_hid)
            xneg = sigmoid( rbm.W' * hneg .+ rbm.vis_bias) .> rand(rbm.n_vis)
        end

        ehp = sigmoid(rbm.W * x + rbm.hid_bias)
        ehn = sigmoid(rbm.W * xneg + rbm.hid_bias)
     
        Delta_W += lr * (kron(x, ehp') - kron(xneg, ehn'))'
        Delta_b += lr * (x - xneg)
        Delta_c += lr * (ehp - ehn)
        #print("\n\nENDED")

    end

    rbm.W += Delta_W / batch_size;
    rbm.vis_bias += Delta_b / batch_size;
    rbm.hid_bias += Delta_c / batch_size;
    
    return 
end

contrastive_divergence_K (generic function with 1 method)

In [15]:
X_batch = X_train[:,1:25];
size(X_batch)

(784,25)

In [16]:
size(X_batch)[2]

25

In [17]:
xneg = X_batch[:,1];
size(xneg)

(784,)

In [18]:
hneg = sigmoid( rbm.W * xneg .+ rbm.hid_bias) .> rand(rbm.n_hid);
size(hneg)

(225,)

In [19]:
xneg = sigmoid(  rbm.W' *hneg  .+ rbm.vis_bias) .> rand(rbm.n_vis)
size(xneg)

(784,)

In [20]:
size(rbm.W)

(225,784)

In [21]:
@time contrastive_divergence_K(X_batch, rbm, 1, 0.01)


batch size: 25  0.790234 seconds (482.80 k allocations: 230.018 MB, 2.16% gc time)


In [218]:
#@benchmark contrastive_divergence_K(X_batch, rbm, 1, 0.01);

In [219]:
size(X_train), size(X_batch)

((784,60000),(784,25))

# Fit RBM

In [33]:
function fit_CDK(X, rbm, batch_size::Integer,  n_epochs::Integer, K::Integer, lr::Real)
        
    n_samples = size(X)[2]
    indicies = [x:min(x + batch_size-1, n_samples) for x in 1:batch_size:n_samples]

    for epoch in 1:n_epochs
        tic();
        for minibatch_ind in indicies
            contrastive_divergence_K(X[:, minibatch_ind], rbm, K, lr)
        end
        print("\nepoch ", epoch, "  time epoch:", toq())
    end
    rbm.trained = true
end

fit_CDK (generic function with 1 method)

In [34]:
n_epochs = 1
batch_size = 200
K = 1
lr = 0.01

fit_CDK(X_train, rbm, batch_size,  n_epochs, K, lr)

LoadError: InterruptException: